# 00_serving_move_file_silver_to_gold_job
---
Este job realiza a copia o dataset agregado `flights_aggregated.parquet` da camada **Silver** para a **Gold**.


In [7]:
# Parameters

run_mode = "latest"
run_date = None

silver_path = "/opt/airflow/data-layer/silver"
gold_path = "/opt/airflow/data-layer/gold"

aggregated_name = "flights_aggregated.parquet"


In [6]:
import os
import shutil
from datetime import datetime
from pathlib import Path

from transformer.utils.spark_helpers import get_spark_session
from transformer.utils.file_io import find_partition
from transformer.utils.logger import get_logger

log = get_logger("serving_move_file_silver_to_gold")

spark = get_spark_session("MoveFileSilverToGold")
log.info("[Serving][Move] SparkSession iniciada.")


2025-11-08 21:32:17 [INFO] serving_move_file_silver_to_gold | [INFO] Logger inicializado no modo standalone (INFO).
2025-11-08 21:32:17 [INFO] spark_helpers | [INFO] SparkSession criada: 'MoveFileSilverToGold' (master=local[*])
2025-11-08 21:32:17 [INFO] serving_move_file_silver_to_gold | [Gold] SparkSession iniciada.


# Executando o job de movimentação

In [ ]:
try:
    log.info("[Serving][Move] Iniciando job de movimentação.")

    # Descobre partição mais recente na silver
    source_partition = find_partition(silver_path, mode=run_mode, date_str=run_date)
    source_dir = Path(silver_path) / source_partition / "PARQUET" / aggregated_name

    if not source_dir.exists():
        raise FileNotFoundError(f"[Serving][Move][ERROR] Arquivo agregado não encontrado: {source_dir}.")

    # Define data de processamento e caminho destino
    processing_date = datetime.now().strftime("%Y-%m-%d")
    dest_dir = Path(gold_path) / processing_date / "PARQUET" / aggregated_name
    dest_dir.parent.mkdir(parents=True, exist_ok=True)

    log.info(f"[Serving][Move] Movendo '{source_dir}' -> '{dest_dir}'.")

    # Cópia do arquivo
    shutil.copytree(source_dir, dest_dir, dirs_exist_ok=True)
    log.info("[Serving][Move] Cópia concluída com sucesso.")

except Exception as e:
    log.exception(f"[Serving][Move][ERROR] Falha durante execução: {e}.")
    raise

finally:
    log.info("[Serving][Move] Job de movimentação encerrado.")


2025-11-08 21:33:03 [INFO] serving_move_file_silver_to_gold | [Serving] Iniciando job de movimentação Silver -> Gold.
2025-11-08 21:33:03 [INFO] serving_move_file_silver_to_gold | [Serving] Movendo '/opt/airflow/data-layer/silver/2025-11-06/PARQUET/flights_aggregated.parquet' -> '/opt/airflow/data-layer/gold/2025-11-08/PARQUET/flights_aggregated.parquet'.
2025-11-08 21:33:06 [INFO] serving_move_file_silver_to_gold | [Serving] Cópia concluída com sucesso.
2025-11-08 21:33:09 [INFO] serving_move_file_silver_to_gold | [Serving] Job de movimentação concluído.
2025-11-08 21:33:09 [INFO] serving_move_file_silver_to_gold | [Serving] Fim do job de movimentação Silver -> Gold.


+-----------+-----------+------------+----------+------------------+-----------+-----------------+----------------+-------------+-----------+------------------------+--------------------+-----------+------------+---------------+----------------+----------------------+--------------------+-----------+----------+-------------+--------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+---------------+-------------+--------+-------+--------+------------+--------------+--------+----------------+--------------+-------------+-------------------+-------------+
|  flight_id|flight_year|flight_month|flight_day|flight_day_of_week|flight_date|airline_iata_code|    airline_name|flight_number|tail_number|origin_airport_iata_code| origin_airport_name|origin_city|origin_state|origin_latitude|origin_longitude|dest_airport_iata_code|   dest_airport_name|  dest_city|dest_state|dest_latitude|dest_longitude|scheduled_departure|    

In [ ]:
%%script false --no-raise-error # Comentar essa linha se estiver em debug ou se quiser rodar a célula.
# Leitura de amostra
df = spark.read.parquet(str(dest_dir))
df.limit(1).show(truncate=True)


In [10]:
# Encerrando a sessão do Spark.
spark.stop()
log.info("[Serving][Move] Sessão Spark finalizada.")


2025-11-08 21:33:12 [INFO] serving_move_file_silver_to_gold | [Serving] Sessão Spark finalizada.
